In [0]:
import torch
import pandas as pd
import numpy as np
import os
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! mkdir covid \
&& cd covid \
&& cp /content/drive/'My Drive'/'Colab Notebooks'/BERT_QA/*.csv .

mkdir: cannot create directory ‘covid’: File exists


In [4]:
!pip install transformers

In [5]:
!wget 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json'
!wget 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin'

--2020-05-16 15:10:40--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.6
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443 [application/json]
Saving to: ‘bert-large-uncased-whole-word-masking-finetuned-squad-config.json.1’

bert-large-uncased- 100%[===================>]     443  --.-KB/s    in 0s      

2020-05-16 15:10:40 (10.1 MB/s) - ‘bert-large-uncased-whole-word-masking-finetuned-squad-config.json.1’ saved [443/443]

--2020-05-16 15:10:42--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.6
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1340675

In [0]:
df1 = pd.read_csv('/content/covid/Covid19_Question_Answer_set.csv',encoding = "ISO-8859-1")
df2 = pd.read_csv('/content/covid/community.csv',encoding = "ISO-8859-1")
df3 = pd.read_csv('/content/covid/news.csv',encoding = "ISO-8859-1")

In [0]:
df1 = pd.DataFrame(df1)

In [8]:
covid_df = df1[['Questions','Answers']]
covid_df = covid_df.replace('\n|\t|\r','', regex=True)
covid_df

,Questions,Answers
0,What is the incubation period of the corona vi...,The incubation period means the time between...
1,Who is most at risk for the corona virus disease?,People of all ages can be infected by the new ...
2,Is the corona virus disease the same as SARS?,No. The virus that causes COVID-19 and the one...
3,What is a corona virus?,corona viruses are a large family of viruses t...
4,Can humans become infected with a novel corona...,Detailed investigations found that SARS-CoV wa...
...,...,...
200,Is it safe to receive a package from any area ...,Yes. The likelihood of an infected person cont...
201,Do I need to wear a mask every time I step out...,Wearing masks does more to protect others agai...
202,What is the government doing to contain the sp...,The government is following the instructions o...
203,"Where can I go to obtain accurate, scientific,...",The websites of the World Health Organization ...


In [0]:
# # community_df = pd.DataFrame(columns=['questions','answer'])
# community_df = df2[['title','answer']]
# community_df = community_df.replace('\n|\t|\r','', regex=True)
# community_df.rename(columns={'title': 'Questions','answer':'Answers'},inplace=True)
# community_df

In [0]:
# news_df  = df3[['question','answer']]
# news_df = news_df.replace('\n|\t|\r','', regex=True)
# news_df.rename(columns={'question': 'Questions','answer':'Answers'},inplace=True)
# news_df

In [0]:
# df = pd.concat([covid_df,community_df,news_df],ignore_index=True)
df = covid_df

In [12]:
df.shape

(205, 2)

In [0]:
import unicodedata
df['Questions'] = df['Questions'].apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())
df['Questions'] = df['Questions'].str.replace('http\S+|www.\S+', '', case=False)
df['Answers'] = df['Answers'].apply(lambda val: unicodedata.normalize('NFKD', str(val)).encode('ascii', 'ignore').decode())
df['Answers'] = df['Answers'].str.replace('http\S+|www.\S+', '', case=False)

In [0]:
context1 = df['Answers'][:20].tolist()
context1 = ' '.join(context1)
with open(r'/content/covid/covid1.txt','w') as wrt:
  wrt.write(context1)
  wrt.close()
# len(context1)

In [0]:
def load_file(source):
    with open(source,'r') as f:
        text = f.read()
        f.close()
    return text


In [0]:
!cp /content/drive/'My Drive'/'Colab Notebooks'/BERT_QA/covid.txt /content/covid/
DATADIR = '/content/covid/'
file = os.path.join(DATADIR,'covid1.txt')
# context = load_file(file)

In [17]:
!pip install pytorch-pretrained-bert

In [0]:
from pytorch_pretrained_bert import BasicTokenizer

In [0]:
from transformers import *
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking-finetuned-squad", do_lower_case=True)
from tqdm import tqdm
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering, BertConfig
from transformers.data.metrics.squad_metrics import _get_best_indexes

In [0]:
def extract_answers(qstring, context, n_answers, max_answer_len,model,config,tokenizer):
    answers = []
    context += ' CANNOTANSWER'
    return run_qa(qstring, context, n_answers, max_answer_len,model,config,tokenizer)

In [0]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_path ='/content/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin'
# config_file = '/content/bert-large-uncased-whole-word-masking-finetuned-squad-config.json'
# config = BertConfig(config_file)
# model = BertForQuestionAnswering(config)
# model.load_state_dict(torch.load(model_path,map_location='cpu'))
# model.to(device)

In [0]:
# Input: Question, paragraph, maximum number of answers to extract, maximum length of the answer 
def run_qa(question, context, nbest, max_answer_len,model,config,tokenizer):
    #Simple sliding window approach for max context cases
    tokenizer_dict = tokenizer.encode_plus(text=question, text_pair=context, max_length=384, stride=128,
                                           return_overflowing_tokens=True, truncation_strategy='only_second',return_token_type_ids=True)
    input_ids = [tokenizer_dict['input_ids']]
    token_type_ids = [tokenizer_dict['token_type_ids']]
    # attention_mask = [tokenizer_dict['attention_mask']]
    # print(tokenizer_dict)
    while 'overflowing_tokens' in tokenizer_dict.keys():
        tokenizer_dict = tokenizer.encode_plus(text=tokenizer.encode(question, add_special_tokens=False), text_pair=tokenizer_dict['overflowing_tokens'], 
                                               max_length=384, stride=128, return_overflowing_tokens=True, truncation_strategy='only_second', 
                                               is_pretokenized=True, pad_to_max_length=True)
        input_ids.append(tokenizer_dict['input_ids'])
        token_type_ids.append(tokenizer_dict['token_type_ids'])    
        # attention_mask.append(tokenizer_dict['attention_mask'])

    all_input_ids = torch.tensor([f for f in input_ids], dtype=torch.long)
    all_token_type_ids = torch.tensor([f for f in token_type_ids], dtype=torch.long)
    # all_attention_mask = torch.tensor([f for f in attention_mask], dtype=torch.long)

    # print(all_input_ids.shape)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_path = model #'/content/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin'
    config_file = config #'/content/bert-large-uncased-whole-word-masking-finetuned-squad-config.json'
    config = BertConfig(config_file)
    model = BertForQuestionAnswering(config)
    model.load_state_dict(torch.load(model_path,map_location='cpu'))
    model.to(device)

    pred_data = TensorDataset(all_input_ids, all_token_type_ids) #,all_attention_mask)
    # Run prediction for full data
    pred_sampler = SequentialSampler(pred_data)
    pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=16)
    # print(len(pred_dataloader.dataset))
    answers = []
    # j = 0
    for input_ids, token_type_ids in tqdm(pred_dataloader):
        model.eval()
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        # attention_mask = attention_mask.to(device)



        with torch.no_grad():
          outputs = model(input_ids, token_type_ids) #,attention_mask) 
        
    
        for i in range(len(input_ids)):
          start_logits, end_logits = [output[i].detach().cpu().tolist() for output in outputs] 
          answers += compute_predictions(start_logits, end_logits, input_ids[i], context.lower(), nbest, max_answer_len,tokenizer)
        # j+=1
    # print(j)
    answers.sort(key = lambda x: len(x['text']), reverse=True)
    # answers.sort(key = lambda x: (x['score'] and len(x['text'])>=8) , reverse=True)
    return answers[0:nbest]

In [0]:

def compute_predictions(start_logits, end_logits, input_ids, context, nbest, max_answer_length,tokenizer):
    start_indexes = _get_best_indexes(start_logits, nbest + 30)
    end_indexes = _get_best_indexes(end_logits, nbest + 30)
    answers = []
    for start_index in start_indexes:
        for end_index in end_indexes:
            #Avoid invalid predictions
            answer_len = end_index - start_index + 1
            if end_index < start_index:
                continue
            if max_answer_length < answer_len:
                continue
            text = tokenizer.decode(input_ids[start_index:start_index + answer_len], clean_up_tokenization_spaces=False)
            try:
                original_start_index = context.index(text)
                original_end_index = original_start_index + len(text)
            except:
                #If there is any problem when looking for the answer in the text
                continue   
            #When answer contains text and cannotanswer remove the cannotanswer part 
            if text != 'cannotanswer':
                text.replace(' cannotanswer', '')

            answer = {'text': text.capitalize(),
                     'score': start_logits[start_index] + end_logits[end_index]
                    }
            # print(type(start_logits[start_index][0]))
            # print(type(end_logits[end_index]))
            answers.append(answer) 
            # break 
    return answers

In [0]:
max_n_answers = 1
# Maximum answer length
max_answer_length = 200
# question = 'What is the incubation period of the corona virus disease?'
# Iterate over all the questions in a task and call the functions above
# df_qa_results = []
context = 'The incubation period means the time between catching the virus and beginning to have symptoms of the disease. Most estimates of the incubation period for COVID-19 range from 1-14 days, most commonly around five days. These estimates will be updated as more data become available. People of all ages can be infected by the new corona virus (2019-nCoV). Older people, and people with pre-existing medical conditions (such as asthma, diabetes, heart disease) appear to be more vulnerable to becoming severely ill with the virus.WHO advises people of all ages to take steps to protect themselves from the virus, for example by following good hand hygiene and good respiratory hygiene. No. The virus that causes COVID-19 and the one that caused the outbreak of Severe Acute Respiratory Syndrome (SARS) in 2003 are related to each other genetically, but the diseases they cause are quite different. corona viruses are a large family of viruses that are known to cause illness ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). Detailed investigations found that SARS-CoV was transmitted from civet cats to humans in China in 2002 and MERS-CoV from dromedary camels to humans in Saudi Arabia in 2012. Several known corona viruses are circulating in animals that have not yet infected humans. As surveillance improves around the world, more corona viruses are likely to be identified. It depends on the virus, but common signs include respiratory symptoms, fever, cough, shortness of breath, and breathing difficulties. In more severe cases, infection can cause pneumonia, severe acute respiratory syndrome, kidney failure and even death. Yes, some corona viruses can be transmitted from person to person, usually after close contact with an infected patient, for example, in a household workplace, or health care centre. When a disease is new, there is no vaccine until one is developed. It can take a number of years for a new vaccine to be developed There is no specific treatment for disease caused by a novel corona virus. However, many of the symptoms can be treated and therefore treatment based on the patients clinical condition. Moreover, supportive care for infected persons can be highly effective. '
# print(question)
# print(extract_answers(question, context, max_n_answers, max_answer_length,'/content/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin','/content/bert-large-uncased-whole-word-masking-finetuned-squad-config.json',tokenizer))


In [0]:
model = '/content/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin'
config = '/content/bert-large-uncased-whole-word-masking-finetuned-squad-config.json'

In [26]:
for i,question in enumerate(df['Questions'][:6]):
  print('Q'+str(i+1)+' : '+question)
  print('Answer: '+extract_answers(question, context, max_n_answers, max_answer_length,model,config,tokenizer)[0]['text'])

Q1 : What is the incubation period of the corona virus disease?


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


Answer: Means the time between catching the virus and beginning to have symptoms of
Q2 : Who is most at risk for the corona virus disease?


100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


Answer: . most estimates of the incubation period for covid
Q3 : Is the corona virus disease the same as SARS?


...

Answer: A large family of viruses that are known to cause illness ranging
Q5 : Can humans become infected with a novel corona virus of animal source?


100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


Answer: , many of the symptoms can be treated and therefore treatment based on the patients clinical
Q6 : What are the symptoms of someone infected with a corona virus?


100%|██████████| 1/1 [00:01<00:00,  1.54s/it]

Answer: , many of the symptoms can be treated and therefore treatment based on the patients clinical
